The first few cells set up phonetisaurus; they are adapted from the instructions in the git README.

In [ ]:
%%capture
!apt-get -y install git g++ autoconf-archive make libtool
# Python bindings
!apt-get -y install python-setuptools python-dev
# mitlm (to build a quick play model)
!apt-get -y install gfortran

In [ ]:
%%capture
!wget http://www.openfst.org/twiki/pub/FST/FstDownload/openfst-1.6.2.tar.gz
!tar -xvzf openfst-1.6.2.tar.gz
%cd openfst-1.6.2
# Minimal configure, compatible with current defaults for Kaldi
!./configure --enable-static --enable-shared --enable-far --enable-ngram-fsts
!make -j 4
# Now wait a while...
!make install

In [17]:
import os
ldlibpath = os.environ['LD_LIBRARY_PATH']
#_STORED_LD = "/usr/local/nvidia/lib:/usr/local/nvidia/lib64"
newld = f'{ldlibpath}:/usr/local/lib:/usr/local/lib/fst'
os.environ['LD_LIBRARY_PATH']=newld
%env LD_LIBRARY_PATH

'/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/lib:/usr/local/lib/fst'

In [ ]:
%%capture
%cd /content
!git clone https://github.com/AdolfVonKleist/Phonetisaurus.git
%cd Phonetisaurus
!./configure
!make
!make install

In [36]:
%cd /content/

/content


We also need MITLM

In [ ]:
%%capture
!git clone https://github.com/mitlm/mitlm
%cd mitlm
!autoreconf -i
!./configure
!make
!make install

In [38]:
%cd /content

/content


The TIMIT dictionary is relatively clean, so there are only a few small changes that are needed for phonetisaurus.

In [82]:
!cat TIMITDIC.txt|grep -v '^;'|tr -d '/'|sed -e 's/  */ /g;s/~adj//;s/~v_past//;s/~v_pres//;s/~v//;s/~n//;' > TIMIT.cleaned
!cat TIMIT.cleaned | perl -pe 's/\s+/ /g; s/^\s+//; s/\s+$//; @_ = split (/\s+/); $w = shift (@_); $_ = $w."\t".join (" ", @_)."\n";' > TIMIT.clean

In [83]:
!phonetisaurus-align --input=TIMIT.clean --ofile=TIMIT.clean.corpus --seq1_del=false
# Train an n-gram model (5s-10s):
!estimate-ngram -o 8 -t TIMIT.clean.corpus -wl timit.arpa
# Convert to OpenFst format (10s-20s):
!phonetisaurus-arpa2wfst --lm=timit.arpa --ofile=timit.fst

GitRevision: 0.9.1
Loading input file: TIMIT.clean
Alignment failed: x
Starting EM...
Finished first iter...
Iteration: 1 Change: 2.70318
Iteration: 2 Change: 0.0603504
Iteration: 3 Change: 0.0425539
Iteration: 4 Change: 0.0206814
Iteration: 5 Change: 0.0114756
Iteration: 6 Change: 0.00711536
Iteration: 7 Change: 0.0042429
Iteration: 8 Change: 0.00297546
Iteration: 9 Change: 0.00223923
Iteration: 10 Change: 0.00151825
Iteration: 11 Change: 0.00115204
Last iteration: 
0.001	Loading corpus TIMIT.clean.corpus...
0.037	Smoothing[1] = ModKN
0.037	Smoothing[2] = ModKN
0.037	Smoothing[3] = ModKN
0.037	Smoothing[4] = ModKN
0.037	Smoothing[5] = ModKN
0.037	Smoothing[6] = ModKN
0.037	Smoothing[7] = ModKN
0.037	Smoothing[8] = ModKN
0.037	Set smoothing algorithms...
0.037	Y 6.063492e-01
0.037	Y 6.304450e-01
0.037	Y 7.305669e-01
0.037	Y 7.950124e-01
0.037	Y 8.524463e-01
0.038	Y 9.033717e-01
0.038	Y 9.355036e-01
0.038	Y 9.092702e-01
0.038	Estimating full n-gram model...
0.040	Saving LM to timit.arpa

In [89]:
!tail TIMIT.clean.corpus

z}z i|n}ih1 n}n i}iy a}ax s}z
z}z i}ih1 p}p
z}z i}ih1 p|p}p e|r}axr s}z
z}z i|r}er1 c}k o}aa n}n s}z
z}z o}aa1 m}m b}b i}iy e}_
z}z o}ow1 n}n e}_ s}z
z}z o}ow1 n}n i}ix n|g}ng
z}z o|o}uw1
z}z o}ow o}aa1 l}l o}ax g}jh i}ix s}s t}t
z}z o|o}uw1 s}z


In [95]:
!cat TIMIT.clean|cut -f 1|tail -n 10 > TIMIT.list

In [96]:
!phonetisaurus-apply --model timit.fst --word_list TIMIT.list

zinnias	z ih1 n iy ax z
zip	z ih1 p
zippers	z ih1 p axr z
zircons	z er1 k aa n z
zombie	z aa1 m b iy
zones	z ow1 n z
zoning	z ow1 n ix ng
zoo	z uw1
zoologist	z ow aa1 l ax jh ix s t
zoos	z uw1 z
